In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

### Data preparationdataset

In [ ]:
import cicids2017
from pathlib import Path
from ProphetStorm import ProphetStorm


RES_PATH = "."
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")


dataset = cicids2017.load_dataset(DATASETPATH_CACHE)
tr = dataset["TR"]

ps = ProphetStorm()
ps.fit(tr)

### Training prophet storm

In [ ]:
import pandas as pd

dt = [dataset["DT"]]

netdata = ps.isanomaly(dt)
Y = pd.concat(dt)["_isanomaly"]
Y = (Y=="none").replace({True: 0, False: 1}).values
y_hat = netdata["_y_hat"].values

In [ ]:
from sklearn import metrics


metrics_rep = [ metrics.roc_auc_score,
                metrics.precision_score, metrics.recall_score,
                metrics.accuracy_score, metrics.f1_score]
for m in metrics_rep:
    mres = m(Y, y_hat)
    print(f"{m.__name__}(moday+attacks): {mres}")

tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
print("\n Confusion matrix")
print(f"\ttp: {tp} \tfp: {fp} \n\tfn: {fn} \ttn: {tn}")